In [1]:
import snowexsql.db
from snowexsql.db import get_db
from snowexsql.data import PointData, SiteData
from snowexsql.conversions import query_to_geopandas

import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime

import os, sys

from PIL import Image, ExifTags
import os
wd = os.getcwd()
print("Current WD: {}".format(wd))

In [36]:
# used another output suprpession found here: https://www.codeforests.com/2020/11/05/python-suppress-stdout-and-stderr/

class suppress_output:
    def __init__(self, suppress_stdout=False, suppress_stderr=False):
        self.suppress_stdout = suppress_stdout
        self.suppress_stderr = suppress_stderr
        self._stdout = None
        self._stderr = None

    def __enter__(self):
        devnull = open(os.devnull, "w")
        if self.suppress_stdout:
            self._stdout = sys.stdout
            sys.stdout = devnull

        if self.suppress_stderr:
            self._stderr = sys.stderr
            sys.stderr = devnull

    def __exit__(self, *args):
        if self.suppress_stdout:
            sys.stdout = self._stdout
        if self.suppress_stderr:
            sys.stderr = self._stderr
            
with suppress_output(suppress_stdout=True, suppress_stderr=False):
    !aws s3 sync --no-progress s3://snowex-data/tutorial-data/camera-trap/ ./cropped

In [7]:
# Connect to the database
db_name = 'snow:hackweek@52.32.183.144/snowex'
engine, session = get_db(db_name)

# Grab all the point data that was that was measured with a camera-trap
qry = session.query(PointData)
qry = qry.filter(PointData.instrument == 'camera-trap')

# Convert it to a geopandas df
camera_depths = query_to_geopandas(qry, engine)

# Grab all the unique pits geometry objects (locations)
qry = session.query(SiteData.geom).distinct()
pits = query_to_geopandas(qry, engine)

# End our database session to avoid hanging transactions
session.close()

In [8]:
from datetime import datetime
# Grab the open site data from the db
open_site = 'W1A'
veg_site = 'W9A'
qry = session.query(PointData).filter(PointData.equipment.contains(open_site))
df_open = query_to_geopandas(qry,engine)

# Grab the vegetated site from the db
qry = session.query(PointData).filter(PointData.equipment.contains(veg_site))
df_veg = query_to_geopandas(qry,engine)

df_veg['datetime'] = [datetime.combine(df_veg['date'][i],df_veg['time'][i]).replace(tzinfo=None) for i in range(len(df_veg))]
df_open['datetime'] = [datetime.combine(df_open['date'][i],df_open['time'][i]).replace(tzinfo=None) for i in range(len(df_open))]
df_tomerge = df_open[['date','datetime','value']]

In [57]:
site_loc = 'W1A'
path, dirs, files = next(os.walk('/'.join([wd,f"cropped/{site_loc}"])))
file_count = len(files)
image_data = []

print(file_count)

659


In [58]:
for i,file in enumerate(files):
    # save image path
    filename = f'/cropped/{site_loc}/'.join([wd,file])
    # open image file and extract metadat
    img = Image.open(filename) 
    exif = { ExifTags.TAGS[k]: v for k, v in img._getexif().items() if k in ExifTags.TAGS }
    # convert date and time to datetime
    exif['DateTime'] = datetime.strptime(exif['DateTime'],'%Y:%m:%d %H:%M:%S')
    # append list of tuples (not sure if this is any more efficient)
    image_data.append((exif['DateTime'].date(), filename, exif['DateTime']))
# create image dataframe    


 
# Shows the image in image viewer
df = pd.DataFrame(image_data,columns=['date','photo_id','datetime']).set_index('date')
df = pd.DataFrame(image_data,columns=['date','photo_id','datetime']).set_index('date')
df.head()

,photo_id,datetime
date,,
2020-04-23,/home/jovyan/cam-learning/contributors/danny/c...,2020-04-23 12:00:00
2019-11-06,/home/jovyan/cam-learning/contributors/danny/c...,2019-11-06 11:00:00
2020-01-06,/home/jovyan/cam-learning/contributors/danny/c...,2020-01-06 11:00:01
2020-01-15,/home/jovyan/cam-learning/contributors/danny/c...,2020-01-15 12:00:01
2020-08-06,/home/jovyan/cam-learning/contributors/danny/c...,2020-08-06 11:00:00


In [59]:
df_snow = df.merge(df_tomerge, on='datetime', how='left')
df_snow = df_snow.rename(columns={'value':'depth'})
df_snow = df_snow[~df_snow['depth'].isnull()]
df_snow['depth'] = df_snow['depth'].clip(lower=0)
df_snow = df_snow.sort_values(by=['date'])
df_snow = df_snow.reindex()
df_snow['photo_id']

6      /home/jovyan/cam-learning/contributors/danny/c...
356    /home/jovyan/cam-learning/contributors/danny/c...
443    /home/jovyan/cam-learning/contributors/danny/c...
145    /home/jovyan/cam-learning/contributors/danny/c...
119    /home/jovyan/cam-learning/contributors/danny/c...
                             ...                        
40     /home/jovyan/cam-learning/contributors/danny/c...
463    /home/jovyan/cam-learning/contributors/danny/c...
615    /home/jovyan/cam-learning/contributors/danny/c...
118    /home/jovyan/cam-learning/contributors/danny/c...
292    /home/jovyan/cam-learning/contributors/danny/c...
Name: photo_id, Length: 338, dtype: object

In [60]:
for filename in os.listdir('/'.join([wd,f'cropped/{site_loc}'])):
    path = f'/cropped/{site_loc}/'.join([wd,filename])
    if path not in list(df_snow['photo_id']):
        os.remove(path)
    else:
        im = Image.open(path)
        width, height = im.size
        # Setting the points for cropped image
        if site_loc == 'W1A':
            left = 3000
            top = 50
            right = 3400
            bottom = 0.63*height
        elif site_loc == 'W9A':
            left = 2100
            top = 150
            right = 2500
            bottom = height - 500
        else: break

        # Cropped image of above dimension
        # (It will not change original image)
        im = im.crop((left, top, right, bottom))
        im.save(path)

In [61]:

    # Size of the image in pixels (size of original image)
    # (This is not mandatory)


ModuleNotFoundError: No module named 'cv2'